In [0]:
#%pip install -r requirements.txt

In [0]:
#dbutils.library.restartPython()

In [0]:
import json
import pandas as pd
from tqdm import tqdm
import webbrowser
import seaborn as sns
import matplotlib.pyplot as plt
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter
from google_play_scraper import Sort, reviews, app
import pycountry
import googletrans
from googletrans import Translator

In [0]:
#%run ./_paramgplayscrapper

In [0]:
#%run ./_envsettings

In [0]:
pycountry.countries.search_fuzzy(fuzzy_search_for_country)

In [0]:
app_infos = []
for ap in tqdm(app_packages):
  info = app(ap, lang='en', country='ae')
  del info['comments']
  app_infos.append(info)
print(app_infos)

In [0]:
def print_json(json_object):
  json_str = json.dumps(
    json_object,
    indent=2,
    sort_keys=True,
    default=str
  )
  
#print(highlight(json_str, JsonLexer(), TerminalFormatter()))

#print_json(app_infos[0])

In [0]:
app_reviews = []

for ap in tqdm(app_packages):
  for score in list(range(1, 6)):
      for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
        rvs, _ = reviews(
          ap,
          lang='en',
          country='ae',
          sort=sort_order,
          #count= 200 if score == 3 else 100,
          #filter_score_with=score
        )
        for r in rvs:
          r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
          r['appId'] = ap
          r['content']
          #print(r)
        app_reviews.extend(rvs)

print(app_reviews)

In [0]:
app_reviews_df = pd.DataFrame(app_reviews)
app_reviews_df = app_reviews_df.drop_duplicates(subset=['content'], keep='first')
print(app_reviews_df)

In [0]:

#Writing to generic output folder
app_reviews_df.to_csv(
    f"abfss://root@ecommerceadlsgen2.dfs.core.windows.net/{abs_file_name}",
    index=False,
    storage_options={"account_name": account_name, "account_key": account_key},
)

In [0]:
#Writing data to parquest file in bronze layer
spark.conf.set(f"fs.azure.account.key.{account_name}.dfs.core.windows.net", account_key)
csv_path  = f"abfss://root@{account_name}.dfs.core.windows.net/{abs_file_name}"
df = spark.read.option("header", "true").option("inferSchema", "true").csv(csv_path)
display(df)
delta_path = f"abfss://root@{account_name}.dfs.core.windows.net/datalake/bronze/{output_location}"
df.write.format("delta").mode("overwrite").save(delta_path)